# Data Cleaning & Preprocessing

## Objectives

* Prepare the electricity cost dataset for machine learning
* Apply data cleaning, preprocessing, and light feature engineering informed by EDA findings
* Ensure that engineered features are model-ready while retaining business interpretability

## Inputs

* outputs/datasets/collection/ElectricityCost.csv

## Outputs

* Cleaned dataset for modelling
* Preprocessing logic reusable in the ML pipeline

## Additional Comments

* This step supports **Business Requirement 2** by ensuring the input data is suitable for training a reliable electricity cost prediction model.

**IMPORTANT NOTE**: This notebook is intended to be run top-to-bottom.
If errors occur due to re-rerunning individual cells, restart the kernel and run all cells.


---

# Change working directory

We need to change the working directory from its current folder to its parent folder
* We access the current directory with os.getcwd()

In [1]:
import os
current_dir = os.getcwd()
current_dir

'/workspaces/PP5-TBC-/jupyter_notebooks'

We want to make the parent of the current directory the new current directory
* os.path.dirname() gets the parent directory
* os.chdir() defines the new current directory

In [2]:
os.chdir(os.path.dirname(current_dir))
print("You set a new current directory")

You set a new current directory


Confirm the new current directory

In [3]:
current_dir = os.getcwd()
current_dir

'/workspaces/PP5-TBC-'

---

# Load data

Loading the dataset produced in the data collection notebook for cleaning and preprocessing.

In [4]:
import pandas as pd
import numpy as np

df = pd.read_csv("outputs/datasets/collection/ElectricityCost.csv")
df.head()

,site area,structure type,water consumption,recycling rate,utilisation rate,air qality index,issue reolution time,resident count,electricity cost
0,1360,Mixed-use,2519.0,69,52,188,1,72,1420.0
1,4272,Mixed-use,2324.0,50,76,165,65,261,3298.0
2,3592,Mixed-use,2701.0,20,94,198,39,117,3115.0
3,966,Residential,1000.0,13,60,74,3,35,1575.0
4,4926,Residential,5990.0,23,65,32,57,185,4301.0


---

# Initial data checks

Confirm dataset shape, data types, and absence of missing values before applying transformations. 

In [5]:
df.shape

(10000, 9)

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 9 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   site area             10000 non-null  int64  
 1   structure type        10000 non-null  object 
 2   water consumption     10000 non-null  float64
 3   recycling rate        10000 non-null  int64  
 4   utilisation rate      10000 non-null  int64  
 5   air qality index      10000 non-null  int64  
 6   issue reolution time  10000 non-null  int64  
 7   resident count        10000 non-null  int64  
 8   electricity cost      10000 non-null  float64
dtypes: float64(2), int64(6), object(1)
memory usage: 703.3+ KB


---

# Standardise column names

Column names are standardised to snake_case for consistency, readability, and compatibility with ML pipelines.
This addresses minor naming inconsistencies identified during EDA.

In [7]:
df.columns = (
    df.columns
    .str.strip()
    .str.lower()
    .str.replace(" ", "_")
)

df.columns

Index(['site_area', 'structure_type', 'water_consumption', 'recycling_rate',
       'utilisation_rate', 'air_qality_index', 'issue_reolution_time',
       'resident_count', 'electricity_cost'],
      dtype='object')

---

# Correct known column name spelling issues

Minor spelling errors are corrected to avoid confusion and ensure clarity in the subsequent analysis.

In [8]:
df = df.rename(columns={
    "air_qality_index": "air_quality_index",
    "issue_reolution_time": "issue_resolution_time"
})

df.columns

Index(['site_area', 'structure_type', 'water_consumption', 'recycling_rate',
       'utilisation_rate', 'air_quality_index', 'issue_resolution_time',
       'resident_count', 'electricity_cost'],
      dtype='object')

---

# Encode structure type

The variable `structure type` is a nominal categorical variable with no inherent order.
One-hot encoding is applied to prepare it for use in regression models.

In [9]:
df = pd.get_dummies(
    df,
    columns=["structure_type"],
    drop_first=True
)

df.head()

,site_area,water_consumption,recycling_rate,utilisation_rate,air_quality_index,issue_resolution_time,resident_count,electricity_cost,structure_type_Industrial,structure_type_Mixed-use,structure_type_Residential
0,1360,2519.0,69,52,188,1,72,1420.0,False,True,False
1,4272,2324.0,50,76,165,65,261,3298.0,False,True,False
2,3592,2701.0,20,94,198,39,117,3115.0,False,True,False
3,966,1000.0,13,60,74,3,35,1575.0,False,False,True
4,4926,5990.0,23,65,32,57,185,4301.0,False,False,True


One-hot encoding converts the structure type variable into multiple binary features that indicate the presence or absence of each category.
This approach avoids introducing any artificial ordering between structure types and ensures the model interprets them purely as distinct categories.

---

# Handle resident count feature

EDA showed that ~40% of sites have zero residents, which is expected for commercial and industrial properties.
To reduce skewness and improve interpretability, resident count is grouped into meaningful ranges.
This step involves feature engineering, as a new categorical feature is derived from the original resident count variable in order to better capture occupancy patterns.

In [10]:
def resident_group(x):
    if x == 0:
        return "none"
    elif x <= 50:
        return "low"
    elif x <= 150:
        return "medium"
    else:
        return "high"

df["resident_group"] = df["resident_count"].apply(resident_group)
df["resident_group"].value_counts()

resident_group
none      4009
high      2364
medium    2275
low       1352
Name: count, dtype: int64

In [11]:
df = pd.get_dummies(
    df,
    columns=["resident_group"],
    drop_first=True
)

This engineered feature allows the model to capture differences in occupancy levels without assuming a linear relationships.

---

# Transform water consumption

In the EDA `water consumption` showed a right-skewed distribution.
A log transformation is applied to stabilise variance and improve model performance.

In [12]:
df["water_consumption_log"] = np.log1p(df["water_consumption"])

The log transformation reduces the impact of extreme values and helps stabilise the distribution of water consumption. 
The original water consumption variable is retained to allow comparison between raw and transformed features during model evaluation.

---

# Feature preparation summary

At this stage:
* Categorical variables have been encoded
* Skewed variables have been transformed
* Original variables are retained where appropriate for later comparison

Final feature scaling will be applied within machine learning pipeline to prevent data leakage.
Train/test splitting is intentionally deferred to the modelling notebook to ensure that all data leakage-sensitive operations are handled within a controlled pipeline.

---

# Final dataset overview

Inspect the dataset after cleaning and preprocessing to confirm readiness for modelling.

In [13]:
df.shape

(10000, 15)

In [14]:
df.head()

,site_area,water_consumption,recycling_rate,utilisation_rate,air_quality_index,issue_resolution_time,resident_count,electricity_cost,structure_type_Industrial,structure_type_Mixed-use,structure_type_Residential,resident_group_low,resident_group_medium,resident_group_none,water_consumption_log
0,1360,2519.0,69,52,188,1,72,1420.0,False,True,False,False,True,False,7.832014
1,4272,2324.0,50,76,165,65,261,3298.0,False,True,False,False,False,False,7.751475
2,3592,2701.0,20,94,198,39,117,3115.0,False,True,False,False,True,False,7.901748
3,966,1000.0,13,60,74,3,35,1575.0,False,False,True,True,False,False,6.908755
4,4926,5990.0,23,65,32,57,185,4301.0,False,False,True,False,False,False,8.698014


In [15]:
df.isna().sum()

site_area                     0
water_consumption             0
recycling_rate                0
utilisation_rate              0
air_quality_index             0
issue_resolution_time         0
resident_count                0
electricity_cost              0
structure_type_Industrial     0
structure_type_Mixed-use      0
structure_type_Residential    0
resident_group_low            0
resident_group_medium         0
resident_group_none           0
water_consumption_log         0
dtype: int64

---

# Save cleaned dataset

The cleaned dataset is saved for use in the modelling notebook.

In [16]:
try:
  os.makedirs(name='outputs/datasets/cleaned')
except Exception as e:
  print(e)

df.to_csv("outputs/datasets/cleaned/ElectricityCostCleaned.csv", index=False)

---

# Conclusions

This notebook transformed the raw electricity cost dataset into a model-ready format.
Key preprocessing steps were guided directly by EDA findings and business context.

* Column naming inconsistencies were resolved
* Categorical variables were encoded appropriately
* Skewed and sparse variables were transformed to improve model stability
* No data leakage was introduced

# Next Steps

The next notebook will focus on:
* Train/test set splitting
* Feature scaling within a pipeline
* Training and evaluating regression models to meet Business Requirement 2 